In [10]:
#r "nuget: MathNet.Numerics"

open System.Linq;
open System.Collections.Generic
open System.Collections;
open MathNet.Numerics.LinearAlgebra
open MathNet.Numerics.Distributions

Installed Packages MathNet.Numerics, 5.0.0

## Domain

In [11]:
type Goal = 
    | Max
    | Min

type GaussianKernel = { LengthScale : double; Variance : double }
type DataPoint      = { X : double; Y : double }
type GaussianProcess = 
    { 
        Kernel: GaussianKernel
        DataPoints: List<DataPoint>
        CovarianceMatrix: Matrix<double>
    }

type AcquistionFunctionResult = { X : double; Y : double }

type EstimationResult =
    { 
        Mean       : double
        LowerBound : double
        UpperBound : double
        X          : double
    }

type GaussianModel =
    {
        GaussianProcess : GaussianProcess 
        Query           : double -> double
        Inputs          : double list
    }

type ModelResult = 
    { 
        AcquistionFunctionResult : AcquistionFunctionResult
        EstimationResult         : EstimationResult
        DataPoint                : DataPoint
    }

## Functions

### Kernel 

In [12]:
// Kernel Method.
let gaussianKernelCompute (kernel: GaussianKernel) (left : double) (right : double) : double  = 
    if left = right then kernel.Variance
    else 
        let squareDistance : double = Math.Pow( left - right, 2 )
        kernel.Variance * Math.Exp( -squareDistance / ( kernel.Variance * kernel.Variance ))

### Addition of New Data Points and Estimation of the Posterior

In [4]:
// Add New Data Point.
//let addNewDataPoint (gaussianProcess : GaussianProcess) (dataPoint : DataPoint) : unit = 

Error: input.fsx (7,5)-(7,33) typecheck error This expression was expected to have type
    'unit'    
but here has type
    ''a list'    

In [13]:
// Estimation Method.
let estimateAtPoint (gaussianProcess : GaussianProcess) (x : double) : EstimationResult = 
    let kStar : double array = 
        gaussianProcess.DataPoints
                       .Select(fun dp -> gaussianKernelCompute gaussianProcess.Kernel dp.X dp.Y)
                       .ToArray()

    let yTrain : double array = 
        gaussianProcess.DataPoints
                       .Select(fun dp -> dp.Y)
                       .ToArray()

    let ks         : Vector<double> = Vector<double>.Build.Dense kStar
    let f          : Vector<double> = Vector<double>.Build.Dense yTrain
    let common     : Vector<double> = gaussianProcess.CovarianceMatrix.Inverse().Multiply ks
    let mu         : double         = common.DotProduct f
    let confidence : double     = Math.Abs(-common.DotProduct ks + gaussianKernelCompute gaussianProcess.Kernel x x)
    { Mean = mu; LowerBound = mu - confidence; UpperBound = mu + confidence; X = x }

In [14]:
// Based on each subsequent iteration, we compute the next query point by referring to the surrogate function 
// that produces the highest acquisition value.
let estimateAtRange (gaussianProcess : GaussianProcess) (X : double list) : EstimationResult list =
    let result = X |> List.map(fun x -> estimateAtPoint gaussianProcess x)
    result

### Acquisition

In [16]:
// Acquisition Method.
let expectedImprovement (gaussianProcess : GaussianProcess) 
                        (estimationResult : EstimationResult) 
                        (goal : Goal) : AcquistionFunctionResult = 
    // TODO: Improve this logic by keeping score of the max / min based on the goal.
    let bestValue : DataPoint = 
        match goal with
        | Goal.Max -> gaussianProcess.DataPoints.MaxBy(fun l -> l.Y)
        | Goal.Min -> gaussianProcess.DataPoints.MinBy(fun l -> l.Y)

    let delta : double = estimationResult.Mean - bestValue.Y
    let sigma : double = estimationResult.UpperBound - estimationResult.LowerBound
    let z     : double = delta / sigma
    let next  : double = delta * Normal.CDF(0, 1, z) + sigma * Normal.PDF(0, 1, z)
    { X = estimationResult.X ; Y = Math.Max(next, 0) }

### Model

In [17]:
let createModel (gaussianProcess  : GaussianProcess) 
                (query            : double -> double) 
                (min              : double) 
                (max              : double)
                (iterations: int) : GaussianModel = 

    // Random Initialization of Inputs.
    let inputs : double list = seq { for i in 0 .. iterations -> 0 } 
                               |> Seq.mapi(fun d i -> min + double i * (max - min) / (double iterations - 1.))
                               |> Seq.toList
    { GaussianProcess = gaussianProcess; Query = query; Inputs = inputs }

let findExtrema (gaussianModel : GaussianModel) (goal : Goal) : ModelResult =

Error: input.fsx (13,1)-(13,78) parse warning Possible incorrect indentation: this token is offside of context started at position (13:1). Try indenting this token further or using standard formatting conventions.
input.fsx (13,78)-(13,78) parse warning Possible incorrect indentation: this token is offside of context started at position (13:1). Try indenting this token further or using standard formatting conventions.
input.fsx (13,1)-(13,78) parse error Incomplete structured construct at or before this point in binding

## References

1. [Gaussian Processes](http://krasserm.github.io/2018/03/19/gaussian-processes/)
2. [Bayesian Optimization From Scratch](https://machinelearningmastery.com/what-is-bayesian-optimization/)

In [9]:
#!about

.NET Interactive© 2020 Microsoft CorporationVersion: 1.0.360602+9bf026dabac44a6256a65168fa93dcb7e2edcca4Library version: 1.0.0-beta.22606.2+9bf026dabac44a6256a65168fa93dcb7e2edcca4Build date: 2022-12-10T01:03:09.8221170Zhttps://github.com/dotnet/interactive
